In [64]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [65]:
hn = 3
class Model(nn.Module):
     
    def __init__(self):
        super().__init__()
        self.stepLength = 1.2
        self.hidden = torch.zeros((1, hn))
        
        self.Whh = nn.Linear(hn, hn)
        self.Whx = nn.Linear(1, hn, bias = False)
        
        self.layer1 = nn.Linear(1 + hn, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 8)
        self.layer4 = nn.Linear(8, 1)
      
    
    def forward_RNN(self, x):
        self.hidden = torch.tanh(self.Whh(self.hidden) + self.Whx(x))
        
        
    def forward_DNN(self, x):
        
        y = torch.cat((self.hidden.repeat(x.shape[0], 1), x), 1)
        
        y = self.layer1(y).clamp(min=0)
        y = self.layer2(y).clamp(min=0)
        y = self.layer3(y).clamp(min=0)
        y = self.layer4(y)
     
        return y*torch.exp(-0.1*x**2)
    
    def sample(self):
        total = 0
        x = torch.Tensor(4*np.random.random((1,1)) - 2)
        psi_old = self.forward_DNN(x)
        
        for i in range(10):
            x_new = x + 1.5*torch.Tensor(2*np.random.random((1,1)) - 1)
            psi_new = self.forward_DNN(x_new)
            
            if (psi_new/psi_old)**2 > np.random.random():
                x = x_new
                psi_old = psi_new
                total += 1
            
        return x, total
    
    def resetHidden(self):
        self.hidden = torch.zeros((1, hn))
        self.forward_RNN(torch.zeros((1, 1)))
        

In [66]:
torch.manual_seed(42)
np.random.seed(42)
model = Model()
optimizer = torch.optim.Adam(model.parameters())

### Using naive minimization of energy

In [ ]:
epochs = 2000
N = 30
for epoch in tqdm(range(epochs)):
    PE_acc = 0
    P_acc = 0
    E_acc = 0
    grad = 0
    
    for i in range(N):
        psi_total = 1
        
        model.resetHidden() 
        x1 = model.sample()[0].detach().requires_grad_()
        psi1 = model.forward_DNN(x1)
        
        model.forward_RNN(x1)
        x2 = model.sample()[0].detach().requires_grad_()
        psi2 = model.forward_DNN(x2)
        
        
        psi_total = psi1*psi2
        dfdx, = torch.autograd.grad(psi_total, x1, create_graph=True)
        d2fdx2, = torch.autograd.grad(dfdx, x1, create_graph=True)
        
        kinetic1 = -0.5*d2fdx2/psi_total
        potential1 = 0.5*x1**2

        
        dfdx, = torch.autograd.grad(psi_total, x2, create_graph=True)
        d2fdx2, = torch.autograd.grad(dfdx, x2, create_graph=True)
        
        kinetic2 = -0.5*d2fdx2/psi_total
        potential2 = 0.5*x2**2
        

        
        E_L = (kinetic1 + potential1 + kinetic2 + potential2 + 0.1/torch.abs(x1-x2)).detach()
        PE_acc += psi_total/psi_total.detach()*E_L
        P_acc  += psi_total/psi_total.detach()
        E_acc  += E_L   
    
    PE_acc /= N
    P_acc  /= N
    E_acc  /= N
    
    L2 = 0
    for param in model.parameters():
        L2 += torch.mean(param**2)
        
    
    loss = 2*(PE_acc - P_acc*E_acc)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    grad = 0
    for param in model.parameters():
        grad += torch.mean(param.grad**2)
    
    
    if epoch%100 == 0: 
        print(f"epoch: {epoch}, Grad: {grad}, Energy: {E_acc.item()}")

  0%|          | 1/2000 [00:00<07:24,  4.49it/s]

epoch: 0, Grad: 302.8515625, Energy: 2.267035484313965


  5%|▌         | 101/2000 [00:21<06:33,  4.83it/s]

epoch: 100, Grad: 1.2957344055175781, Energy: 0.8138153553009033


  8%|▊         | 161/2000 [00:33<06:42,  4.57it/s]

In [71]:
def f(x):
    x_lin = torch.linspace(-5, 5, 100).reshape(100,-1)

    model.resetHidden()
    psi1 = model.forward_DNN(x_lin)[:,0].detach().numpy()

    x = torch.Tensor([[x]])
    #y = torch.Tensor([[y]])
    
    model.forward_RNN(x)
    psi2 = model.forward_DNN(x_lin)[:,0].detach().numpy()

    #model.forward_RNN(y)
    #psi3 = model.forward_DNN(x_lin)[:,0].detach().numpy()

    plt.plot(x_lin[:,0], psi1**2/np.sum(psi1**2), "b")
    plt.plot(x_lin[:,0], psi2**2/np.sum(psi2**2), "r")
    #plt.plot(x_lin[:,0], psi3**2/np.sum(psi3**2))
    plt.plot(x, 0.01, "bo")
    #plt.plot(y, 0.01, "ro")
    plt.xlim((-5,5))
    plt.ylim((0, 0.1))

    plt.show()
    
def g(x):
    model.resetHidden()

    x = torch.Tensor([[x]])
    #y = torch.Tensor([[y]])

    plt.plot(model.hidden.detach().numpy(), "bo")
    model.forward_RNN(x)
    #model.forward_RNN(y)

    plt.plot(model.hidden.detach().numpy(), "ro")
    plt.xlim((-5,5))
    plt.ylim((-1, 1))

    plt.show()

interact(f, x=(-4.0, 4., 0.1));
interact(g, x=(-4.0, 4., 0.1));

interactive(children=(FloatSlider(value=0.0, description='x', max=4.0, min=-4.0), Output()), _dom_classes=('wi…

interactive(children=(FloatSlider(value=0.0, description='x', max=4.0, min=-4.0), Output()), _dom_classes=('wi…

In [18]:
model.resetHidden()
print(model.sample())

(tensor([[1.2257]]), 7)


In [ ]:
x_lin = torch.linspace(-4, 4, 100).reshape(100,-1)
psi = model.forward(x_lin)[:,0].detach().numpy()

plt.plot(x_lin[:,0], psi**2)
plt.plot(x_lin[:,0], 1/np.sqrt(np.pi)*np.exp(-x_lin[:,0]**2), "--")
plt.legend(["DNN", "analytical"])
plt.show()


## Estimating energy

In [36]:
N = 10000

E = 0
for i in tqdm(range(N)):
    model.resetHidden() 
    x1 = model.sample()[0].detach().requires_grad_()
    psi1 = model.forward_DNN(x1)

    model.forward_RNN(x1)
    x2 = model.sample()[0].detach().requires_grad_()
    psi2 = model.forward_DNN(x2)

    psi_total = psi1*psi2
    dfdx, = torch.autograd.grad(psi_total, x1, create_graph=True)
    d2fdx2, = torch.autograd.grad(dfdx, x1, create_graph=True)

    kinetic1 = -0.5*d2fdx2/psi_total
    potential1 = 0.5*x1**2

    dfdx, = torch.autograd.grad(psi_total, x2, create_graph=True)
    d2fdx2, = torch.autograd.grad(dfdx, x2, create_graph=True)

    kinetic2 = -0.5*d2fdx2/psi_total
    potential2 = 0.5*x2**2

    E += (kinetic1 + potential1 + kinetic2 + potential2 + 0/torch.abs(x1-x2)).detach()

E = E/N

print(E.item())

100%|██████████| 10000/10000 [01:04<00:00, 154.26it/s]

1.1279858350753784


## Testing auto grad

In [ ]:
x = [torch.Tensor([3]).requires_grad_(), torch.Tensor([6]).requires_grad_()]
y = torch.Tensor([4, 8]).requires_grad_()

f = torch.Tensor(x)**3*y**2

print(f)
#f.backward()
#print(x.grad, y.grad)

dfdx = [torch.autograd.grad(a, b, create_graph=True)[0] for a, b in zip(f, x)]
print(dfdx)

d2fdx2 = [torch.autograd.grad(a, b, create_graph=True)[0] for a, b in enumerate(dfdx, x)]

print(d2fdx2)

#d2fdx2.backward()
#print(x.grad, y.grad)

#print(dfdx)


#ddfdxdx, = torch.autograd.grad(dfdx, x, create_graph=True)
#print(ddfdxdx.grad_fn(x)[1])